In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import numpy.random as rnd

In [3]:
dat = tf.data.Dataset.from_tensor_slices(
    tensors = {
        "name": ["a", "b", "c"],
        "x": [1, 2, 3],
        "y": rnd.rand(3, 2),
    }
)

In [4]:
dat

<TensorSliceDataset shapes: {y: (2,), x: (), name: ()}, types: {y: tf.float64, x: tf.int32, name: tf.string}>

In [11]:
dat.map(lambda vv: vv["x"])

<MapDataset shapes: (), types: tf.int32>

In [12]:
dat

<TensorSliceDataset shapes: {x: (), name: (), y: (2,)}, types: {x: tf.int32, name: tf.string, y: tf.float64}>

In [14]:
sess = tf.InteractiveSession()

In [9]:
dat.batch(1)

<BatchDataset shapes: {y: (?, 2), x: (?,), name: (?,)}, types: {y: tf.float64, x: tf.int32, name: tf.string}>